Use EN samples from covost2 data (all data except for the pairs that has DE, just in case)

In [1]:
import pandas as pd
import numpy as np
import nltk
import os
import tarfile



np.random.seed(0)



original_data = pd.DataFrame(columns=['SRC'])

data_dir = "data/covost2/EN-translations"

deduplicated_each_dataset = 0

for filename in os.listdir(data_dir):
    if filename.endswith(".tar.gz") and ('de' not in filename):
        unzipped_file_name = filename.replace(".tar.gz", "")
        
        # Extract the file if not yet done so
        if not os.path.exists(os.path.join(data_dir, unzipped_file_name)):
            tar = tarfile.open(os.path.join(data_dir, filename))
            tar.extractall(data_dir)
            tar.close()
            
        tmp_df = pd.DataFrame()
        tmp_df['SRC'] = pd.read_csv(os.path.join(data_dir, unzipped_file_name), sep='\t')['translation']

        deduplicated_each_dataset = deduplicated_each_dataset + tmp_df.drop_duplicates(subset='SRC').shape[0]
        
        original_data = pd.concat([original_data, tmp_df], axis=0, ignore_index=True)
        
        
        
print(f"all: {original_data.shape}")
print(f"deduplicated_each_dataset: {deduplicated_each_dataset}")
print(f"deduplicated all: {original_data.drop_duplicates(subset='SRC').shape}")

original_data = original_data.drop_duplicates(subset='SRC')


all: (312409, 1)
deduplicated_each_dataset: 108694
deduplicated all: (108453, 1)


Filter out the errornously long sentences

In [2]:
sentence_lengths = original_data['SRC'].apply(lambda x: len(x))

length_stats = sentence_lengths.describe(percentiles=[.25, .5, .75, .99])

original_data = original_data[sentence_lengths < length_stats['99%']]

original_data.reset_index(drop=True, inplace=True)

original_data.shape

(107358, 1)

### Perform stemming on the data

This would help reduce the vocab size, easier to later on choose the word to perturb

ABORT: it reduce the vocab from 88066 to 71362, so not that much, so doesnt worth it. Also stemming makes the word invalid, so cannot use POS afterward to filter it out.

Lemmatization would require defining POS --> not preferable, since we would want chinese and china to be the same

In [3]:
# def stem_sentence(stemmer, sentence):
#     """ 
#     Return the stemmed sentence and 
#     a dictionary mapping the stem to the original word in the sentence
#     """
#     tokenized_sentence = nltk.word_tokenize(sentence)
#     stem_word_dict = {}
#     stemmed_tokenized_sentence = []
    
#     for word in tokenized_sentence:
#         stem = stemmer.stem(word)
#         stemmed_tokenized_sentence.append(stem)
#         stem_word_dict[stem] = word
        
#     return ' '.join(stemmed_tokenized_sentence), stem_word_dict
    

In [4]:
# from nltk.stem.snowball import SnowballStemmer

# stemmer = SnowballStemmer(language='english')

# original_data['StemSRC'], original_data['StemDict'] = \
#     zip(*original_data.apply(lambda x: stem_sentence(stemmer, x['SRC']), axis=1))




### Invesitigate in the frequencies of words across sentences

Count the number of occurance in sentence of each word. Here we **only use the sentences where the words only occurs 1 time**, which is convenient to analyse on the influence of the word on the sentences.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import find, csr_matrix

corpus = original_data['SRC'].values
vectorizer = CountVectorizer()
count_fit = vectorizer.fit_transform(corpus)

# Only consider the single occurance of a word in a sentence
count_fit[count_fit > 1] = 0

count_fit.shape

(107358, 46970)

In [6]:
import spacy
import time 


word_df = pd.DataFrame()
word_df['word'] = vectorizer.get_feature_names_out()
word_df['freq'] = np.asarray(count_fit.sum(axis=0)).flatten()


spacy_model = spacy.load("en_core_web_sm")

def spacy_pos_tag(word, spacy_model):
    doc = spacy_model(word)
    return [t.pos_ for t in doc][0]

def nltk_pos_tag(word):
    return nltk.pos_tag([word])[0][1]

def get_entity_name(word, spacy_model):
    """
    Function returning the NER output from spacy on a word
    Return None if the word does not have any entity name
    Labels and there descriptions:
    ```
    nlp = spacy.load("en_core_web_sm")
    labels = nlp.get_pipe('ner').labels
    for label in labels:
        print(f'{label}: {spacy.explain(label)}')
    ```
        CARDINAL: Numerals that do not fall under another type
        DATE: Absolute or relative dates or periods
        EVENT: Named hurricanes, battles, wars, sports events, etc.
        FAC: Buildings, airports, highways, bridges, etc.
        GPE: Countries, cities, states
        LANGUAGE: Any named language
        LAW: Named documents made into laws.
        LOC: Non-GPE locations, mountain ranges, bodies of water
        MONEY: Monetary values, including unit
        NORP: Nationalities or religious or political groups
        ORDINAL: "first", "second", etc.
        ORG: Companies, agencies, institutions, etc.
        PERCENT: Percentage, including "%"
        PERSON: People, including fictional
        PRODUCT: Objects, vehicles, foods, etc. (not services)
        QUANTITY: Measurements, as of weight or distance
        TIME: Times smaller than a day
        WORK_OF_ART: Titles of books, songs, etc.
    """
    
    doc = spacy_model(word)
    for w in doc.ents:
        return w.label_

start = time.time()
word_df['POS'] = word_df['word'].apply(lambda x: nltk_pos_tag(x))
print(f"POS tagging execution time: {time.time() - start} seconds")


POS tagging execution time: 4.194782018661499 seconds


Have a look at the most frequent content words

In [7]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 9999999)

def is_content_tag(nltk_pos):
    content_tags_prefix = ['NN'] #, 'V', 'JJ', 'PRP']  # Noun, verb, adj, adv (RB, but removed), pronoun
    for prefix in content_tags_prefix:
        if nltk_pos.startswith(prefix):
            return True
    return False

def is_stopword(word):
    # Manually define some stopwords (words that dont contain much content, or errornous)
    stopwords = ['is', 'are', 'was', 'were', 'am', 'be', 
                 'not', 'let',
                 'have', 'has', 'had', 
                 'de', 'la', 'du', 're', 'sur', 'des', 'le', 'll', 
                 'oh', 'lot', 'les', 'ah', 'en', 've',
                 'didn', 'bois']
    return word in stopwords


content_word_bool = word_df['POS'].apply(lambda x: is_content_tag(x)) \
    & (~word_df['word'].apply(lambda x: is_stopword(x)))

word_df[content_word_bool].sort_values(
    by='freq', ascending=False
).tail(10)



,word,freq,POS
27209,microlenecamptus,0,NN
8665,chooz,0,NN
24007,langfang,0,NN
28865,narsingdi,0,NN
28864,narrows,0,NNS
18933,guanosine,0,NN
3933,banco,0,NN
41119,talbot,0,NN
28812,nanshu,0,NN
29917,ohlungen,0,NN


### Create input data where we mask a set of words

#### Set of regional words 

In [8]:
# def is_regional_tag(spacy_ner):
#     regional_tags = ['GPE', 'LANGUAGE', 'NORP']
#     return spacy_ner in regional_tags


# start = time.time()
# word_df['NER'] = word_df['word'].apply(lambda x: get_entity_name(x, spacy_model))
# print(f"NER execution time: {time.time() - start} seconds")


# regional_word_bool = word_df['NER'].apply(lambda x: is_regional_tag(x)) \
#     & (~word_df['word'].apply(lambda x: is_stopword(x)))

#### Set of all content words that is frequent in the inference data

We select the words that appears in over 50 sentences

In [9]:
NR_OF_SENTENCES = 50

In [10]:
word_df[content_word_bool][word_df['freq'] > NR_OF_SENTENCES].sort_values('freq', ascending=False).head()


/var/folders/28/10_vywcj3lb14yk3sn91jn3c0000gn/T/ipykernel_6819/3411812722.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_df[content_word_bool][word_df['freq'] > NR_OF_SENTENCES].sort_values('freq', ascending=False).head()


,word,freq,POS
40182,street,3600,NN
29620,number,2171,NN
13257,don,1858,NN
41866,thousand,1855,NN
2012,amendment,1848,NN


In [11]:
filtered_word_bool = content_word_bool & (word_df['freq'] > NR_OF_SENTENCES)

In [12]:
sum(filtered_word_bool)

1125

In [13]:
import re

def mask_sentence(sentence, masked_word):
    """
        sentence: the original sentence without preprocessing
        masked_word: the word to be masked (in lowercase)
    """
    
    # Find the location of the word in the sentence
    word_locations = [m.start() for m in re.finditer(masked_word, sentence.lower())]
    
    # Make sure that it is actually a standalone word (e.g., 'HE' and not 'tHE')
    final_word_location = None
    for x in word_locations:
        # Make sure the character before and after the word is not alphabet
        if (x == 0 or (not sentence.lower()[x-1].isalpha())) and \
            (x + len(masked_word) == len(sentence) or (not sentence.lower()[x + len(masked_word)].isalpha())):
            final_word_location = x
            break
    
    assert final_word_location is not None
    
    return sentence[:final_word_location] + '[MASK]' + sentence[final_word_location+len(masked_word):]


In [14]:
masked_data = pd.DataFrame(columns=['SRC', 'SRC_masked', 'original_word'])

filtered_word_df = word_df[filtered_word_bool]

for word_index, filtered_word_row in filtered_word_df.iterrows():
    # Indices of the sentences that contains the word
    sentence_indices = count_fit.transpose()[word_index].nonzero()[1]
    
    # Randomly select a fixed number of sentences
    sentence_indices = np.random.choice(a=sentence_indices, 
                                        size=NR_OF_SENTENCES, 
                                        replace=False)
    
    # Create a temporary df to store the sentences for this word
    tmp_df = pd.DataFrame()
    tmp_df['SRC'] = original_data.loc[sentence_indices, 'SRC'].values
    tmp_df['original_word'] = filtered_word_row['word']
    
    # Mask the word in those sentences
    tmp_df['SRC_masked'] = \
        original_data.loc[sentence_indices, 'SRC'].apply(
        lambda x: mask_sentence(sentence=x, masked_word=filtered_word_row['word'])
        ).values
    
    # Concat to the whole df
    masked_data = pd.concat([masked_data, tmp_df], axis=0, ignore_index=True)
    
    

In [15]:
masked_data.shape

(56250, 3)

In [16]:
masked_data.head()

,SRC,SRC_masked,original_word
0,“Don’t accept.”,“Don’t [MASK].”,accept
1,"Please forgive us, but we would systematically come back to these proposals even if you do not accept them.","Please forgive us, but we would systematically come back to these proposals even if you do not [MASK] them.",accept
2,Accept the sanitary logic.,[MASK] the sanitary logic.,accept
3,I accept? I can not,I [MASK]? I can not,accept
4,Be brave and proud enough to accept them.,Be brave and proud enough to [MASK] them.,accept


In [17]:
masked_data.to_csv('data/masked_content_covost2_for_en2de.csv')